# Notebook for visualizations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, layout
from bokeh.models import ColumnDataSource, Span, FactorRange, CDSView, GroupFilter
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral6, Spectral5, Viridis256, Colorblind, Magma256, Turbo256
from IPython.display import display, Math
from scipy import optimize
from math import exp, cos, sqrt, pi, sin, sqrt
from typing import List

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

mypalette = sns.color_palette(['#51ffda']) # https://www.hsluv.org/
sns.set_palette(mypalette)

output_notebook()

In [ ]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

## Visualization of curve fitting

In [ ]:
ospitaletto = AVAILABLE_DATASETS[OspitalettoDataset.OSPITALETTO]

miami = AVAILABLE_DATASETS[NOAA2010Dataset.MIAMI_FL].reset_index(drop=False)
fresno = AVAILABLE_DATASETS[NOAA2010Dataset.FRESNO_CA].reset_index(drop=False)
olympia = AVAILABLE_DATASETS[NOAA2010Dataset.OLYMPIA_WA].reset_index(drop=False)
rochester = AVAILABLE_DATASETS[NOAA2010Dataset.ROCHESTER_NY].reset_index(drop=False)

london = AVAILABLE_DATASETS[InsPireDataset.LONDON_UK].reset_index(drop=False)
madrid = AVAILABLE_DATASETS[InsPireDataset.MADRID_SPA].reset_index(drop=False)
rome = AVAILABLE_DATASETS[InsPireDataset.ROME_IT].reset_index(drop=False)
stuttgart = AVAILABLE_DATASETS[InsPireDataset.STUTTGART_GER].reset_index(drop=False)

In [ ]:
ospitaletto["city_key"] = "ospitaletto"
ospitaletto_source = ColumnDataSource(ospitaletto)

us_data = miami.append(other=fresno).append(other=olympia).append(other=rochester)
us_source = ColumnDataSource(us_data)

europe_data = london.append(other=madrid).append(other=rome).append(other=stuttgart)
europe_source = ColumnDataSource(europe_data)

In [ ]:
miami_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.MIAMI_FL)])
fresno_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.FRESNO_CA)])
olympia_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.OLYMPIA_WA)])
rochester_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.ROCHESTER_NY)])

london_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.LONDON_UK)])
madrid_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.MADRID_SPA)])
rome_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.ROME_IT)])
stuttgart_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.STUTTGART_GER)])

In [ ]:
color_map = {
    "miami": "navy",
    "fresno": "red",
    "olympia": "green",
    "rochester": "gray",
    
    "london": "navy",
    "madrid": "red",
    "rome": "green",
    "stuttgart": "gray",
}

## Air Temperature vs Fitted Air Temperature in USA

In [ ]:
max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view)
miami_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view, size=1)


fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                 title="Fresno, California, USA",
                                 y_range=miami_air_temp_and_fit.y_range,
                                 x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
fresno_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["fresno"], alpha=0.4, source=us_source, view=fresno_view, size=1)

olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                  title="Olympia, Washington, USA",
                                  y_range=miami_air_temp_and_fit.y_range,
                                  x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
olympia_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["olympia"], alpha=0.4, source=us_source, view=olympia_view, size=1)

rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                    title="Rochester, New York, USA",
                                    y_range=miami_air_temp_and_fit.y_range,
                                    x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
rochester_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["rochester"], alpha=0.4, source=us_source, view=rochester_view, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                           title="Adjusted air temperature",
                           y_range=miami_air_temp_and_fit.y_range,
                           x_range=miami_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Miami", color=color_map["miami"], alpha=0.7, source=us_source, view=miami_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Fresno", color=color_map["fresno"], alpha=0.7, source=us_source, view=fresno_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Olympia", color=color_map["olympia"], alpha=0.7, source=us_source, view=olympia_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Rochester", color=color_map["rochester"], alpha=0.7, source=us_source, view=rochester_view, line_width=3)

show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [only_air_temp_fit],
]))

## Air Temperature vs Fitted Air Temperature in Europe

In [ ]:
max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, United Kingdom",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
london_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view, size=1)

madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
madrid_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view, size=1)

rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
rome_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view, size=1)

stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", legend_label="Fit Air Temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="London, UK", color=color_map["london"], alpha=0.7, source=europe_source, view=london_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.7, source=europe_source, view=stuttgart_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.7, source=europe_source, view=rome_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="air_temp_fit", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.7, source=europe_source, view=madrid_view, line_width=3)

show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],
    [only_air_temp_fit],
]))

## Visualization of Air Temp and Ground Temp in USA

In [ ]:
max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",
                                y_range=(min_temp, max_temp),)
miami_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)
miami_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["miami"], alpha=0.1, source=us_source, view=miami_view, size=1)


fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Fresno, California, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
fresno_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["fresno"], alpha=0.1, source=us_source, view=fresno_view, size=1)


olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Olympia, Washington, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
olympia_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["olympia"], alpha=1, source=us_source, view=olympia_view, size=1)


rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Rochester, New York, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
rochester_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["rochester"], alpha=0.1, source=us_source, view=rochester_view, size=1)


ground_temp = figure(plot_width=1200, 
                     plot_height=300, 
                     x_axis_type="datetime",
                     title="Estimated Ground Temperature",
                     y_range=london_air_temp_and_fit.y_range,
                     x_range=london_air_temp_and_fit.x_range)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Miami, FL", color=color_map["miami"], alpha=0.7, source=us_source, view=miami_view, line_width=3)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Fresno, CA", color=color_map["fresno"], alpha=0.7, source=us_source, view=fresno_view, line_width=3)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Olympia, WA", color=color_map["olympia"], alpha=0.7, source=us_source, view=olympia_view, line_width=3)
ground_temp.line(x="timestamp", y="ground_temp", legend_label="Rochester, NY", color=color_map["rochester"], alpha=0.7, source=us_source, view=rochester_view, line_width=3)


show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
    [ground_temp],
]))

## Visualization of Air Temp, GroundTemp in Europe

In [ ]:
max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="London, United Kingdom",
                                y_range=(min_temp, max_temp),)
london_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
london_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view, size=1)

madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
madrid_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view, size=1)

rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
rome_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view, size=1)

stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.line(x="timestamp", y="air_temp", legend_label="Air Temp", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
stuttgart_air_temp_and_fit.circle(x="timestamp", y="ground_temp", legend_label="Ground Temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view, size=1)


only_air_temp_fit = figure(plot_width=1200, 
                           plot_height=300, 
                           x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="London, UK", color=color_map["london"], alpha=0.7, source=europe_source, view=london_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.7, source=europe_source, view=stuttgart_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.7, source=europe_source, view=rome_view, line_width=3)
only_air_temp_fit.line(x="timestamp", y="ground_temp", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.7, source=europe_source, view=madrid_view, line_width=3)

show(layout([
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],
    [only_air_temp_fit],
]))

## Visualization of Aquifer and Yearly Mean Temp in USA

In [ ]:
cities = ["Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
stats = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [miami.aquifer_temp.head(1).item(), 
                          fresno.aquifer_temp.head(1).item(),
                          olympia.aquifer_temp.head(1).item(), 
                          rochester.aquifer_temp.head(1).item()],
         'Mean Air Temp': [miami.air_temp.mean(), 
                           fresno.air_temp.mean(),
                           olympia.air_temp.mean(), 
                           rochester.air_temp.mean()],}

x=[(city, stat) for city in cities for stat in stats]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["miami"], color_map["miami"], 
                                           color_map["fresno"], color_map["fresno"],
                                           color_map["olympia"], color_map["olympia"], 
                                           color_map["rochester"], color_map["rochester"]],
                                   alpha=[0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,]))

aquifer_temp = figure(plot_width=1200, 
                      plot_height=600, 
                      x_range=FactorRange(*x), 
                      title="Comparison of aquifer and mean air temperatures across cities in the USA")
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None

show(aquifer_temp)

## Visualization of Aquifer and Yearly Mean Temp in Europe

In [ ]:
cities = ["London, UK", "Stuttgart, Germany", "Madrid, Spain", "Rome, Italy"]
stats = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [london.aquifer_temp.head(1).item(), 
                          stuttgart.aquifer_temp.head(1).item(),
                          madrid.aquifer_temp.head(1).item(), 
                          rome.aquifer_temp.head(1).item()],
         'Mean Air Temp': [london.air_temp.mean(), 
                           stuttgart.air_temp.mean(),
                           madrid.air_temp.mean(), 
                           rome.air_temp.mean()],}

x = [(city, stat) for city in cities for stat in stats]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["london"], color_map["london"], 
                                           color_map["stuttgart"], color_map["stuttgart"],
                                           color_map["madrid"], color_map["madrid"], 
                                           color_map["rome"], color_map["rome"]],
                                   alpha=[0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,
                                          0.7, 0.4,]))

aquifer_temp = figure(plot_width=1200, 
                      plot_height=600, 
                      x_range=FactorRange(*x), 
                      title="Comparison of aquifer and mean air temperatures across cities in Europe")
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None

show(aquifer_temp)

## Visualization of Heat, SW, and DHW Consumption in the USA

In [ ]:
print(us_data.columns)
us_data[["space_heating_temp", "hot_water_temp", "user_temp"]]

In [ ]:
miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                x_axis_type="datetime",
                                title="Miami, Florida, USA",)
miami_air_temp_and_fit.square(x="timestamp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view, size=1)
miami_air_temp_and_fit.circle(x="timestamp", y="SH_consumption", legend_label="SH Consumption", color=color_map["miami"], alpha=0.1, source=us_source, view=miami_view, size=1)
miami_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["miami"], alpha=0.1, source=us_source, view=miami_view, size=1)



fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 x_axis_type="datetime",
                                title="Fresno, California, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.square(x="timestamp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view, size=1)
fresno_air_temp_and_fit.circle(x="timestamp", y="SH_consumption", legend_label="SH Consumption", color=color_map["fresno"], alpha=0.1, source=us_source, view=fresno_view, size=1)
fresno_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["fresno"], alpha=0.1, source=us_source, view=fresno_view, size=1)


olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  x_axis_type="datetime",
                                title="Olympia, Washington, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.square(x="timestamp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view, size=1)
olympia_air_temp_and_fit.circle(x="timestamp", y="SH_consumption", legend_label="SH Consumption", color=color_map["olympia"], alpha=1, source=us_source, view=olympia_view, size=1)
olympia_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["olympia"], alpha=1, source=us_source, view=olympia_view, size=1)


rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    x_axis_type="datetime",
                                title="Rochester, New York, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.square(x="timestamp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view, size=1)
rochester_air_temp_and_fit.circle(x="timestamp", y="SH_consumption", legend_label="SH Consumption", color=color_map["rochester"], alpha=0.1, source=us_source, view=rochester_view, size=1)
rochester_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["rochester"], alpha=0.1, source=us_source, view=rochester_view, size=1)


show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
]))

## Visualization of Total Heat Consumption VS Air Temperature in the USA

In [ ]:
print(us_data.columns)
us_data[["space_heating_temp", "hot_water_temp", "user_temp"]]

In [ ]:
miami_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                #x_axis_type="datetime",
                                title="Miami, Florida, USA",)
miami_air_temp_and_fit.line(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view, line_width=1)
#miami_air_temp_and_fit.circle(x="timestamp", y="air_temp", legend_label="Air Temperature", color=color_map["miami"], alpha=0.1, source=us_source, view=miami_view, size=1)
#miami_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["miami"], alpha=0.1, source=us_source, view=miami_view, size=1)



fresno_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                 #x_axis_type="datetime",
                                title="Fresno, California, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
fresno_air_temp_and_fit.line(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view, line_width=1)
#fresno_air_temp_and_fit.circle(x="timestamp", y="air_temp", legend_label="Air Temperature", color=color_map["fresno"], alpha=0.1, source=us_source, view=fresno_view, size=1)
#fresno_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["fresno"], alpha=0.1, source=us_source, view=fresno_view, size=1)


olympia_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                  #x_axis_type="datetime",
                                title="Olympia, Washington, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
olympia_air_temp_and_fit.line(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view, line_width=1)
#olympia_air_temp_and_fit.circle(x="timestamp", y="air_temp", legend_label="Air Temperature", color=color_map["olympia"], alpha=1, source=us_source, view=olympia_view, size=1)
#olympia_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["olympia"], alpha=1, source=us_source, view=olympia_view, size=1)


rochester_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                    #x_axis_type="datetime",
                                title="Rochester, New York, USA",
                                y_range=miami_air_temp_and_fit.y_range,
                                x_range=miami_air_temp_and_fit.x_range)
rochester_air_temp_and_fit.line(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view, line_width=1)
#rochester_air_temp_and_fit.circle(x="timestamp", y="air_temp", legend_label="Air Temperature", color=color_map["rochester"], alpha=0.1, source=us_source, view=rochester_view, size=1)
#rochester_air_temp_and_fit.triangle(x="timestamp", y="DHW_consumption", legend_label="DHW Consumption", color=color_map["rochester"], alpha=0.1, source=us_source, view=rochester_view, size=1)


show(layout([
    [miami_air_temp_and_fit, fresno_air_temp_and_fit,],
    [olympia_air_temp_and_fit, rochester_air_temp_and_fit],
]))

## Visualization of Total Heat Consumption VS Air Temperature in the Europe

In [ ]:
print(europe_data.columns)
europe_data

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy import stats

london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK) & 
                          (europe_data["air_temp"] <= 15.0)]
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER) & 
                          (europe_data["air_temp"] <= 15.0)]
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT) & 
                          (europe_data["air_temp"] <= 15.0)]
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA) & 
                          (europe_data["air_temp"] <= 15.0)]

london_slope, london_intercept, london_r_value, london_p_value, london_std_err = stats.linregress(london_data["air_temp"], 
                                                                                                  london_data["Total_consumption"])
london_regline = london_slope * london_data["air_temp"] + london_intercept

stuttgart_slope, stuttgart_intercept, stuttgart_r_value, stuttgart_p_value, stuttgart_std_err = stats.linregress(stuttgart_data["air_temp"], 
                                                                                                  stuttgart_data["Total_consumption"])
stuttgart_regline = stuttgart_slope * stuttgart_data["air_temp"] + stuttgart_intercept

rome_slope, rome_intercept, rome_r_value, rome_p_value, rome_std_err = stats.linregress(rome_data["air_temp"], 
                                                                                                  rome_data["Total_consumption"])
rome_regline = rome_slope * rome_data["air_temp"] + rome_intercept

madrid_slope, madrid_intercept, madrid_r_value, madrid_p_value, madrid_std_err = stats.linregress(madrid_data["air_temp"], 
                                                                                                  madrid_data["Total_consumption"])
madrid_regline = madrid_slope * madrid_data["air_temp"] + madrid_intercept



london_air_temp_and_fit = figure(plot_width=600, 
                                plot_height=300, 
                                title="London, United Kingdom",)
london_air_temp_and_fit.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
london_air_temp_and_fit.line(x=london_data["air_temp"], y=london_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


madrid_air_temp_and_fit = figure(plot_width=600,  
                                 plot_height=300, 
                                title="Madrid, Spain",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
madrid_air_temp_and_fit.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
madrid_air_temp_and_fit.line(x=madrid_data["air_temp"], y=madrid_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


rome_air_temp_and_fit = figure(plot_width=600, 
                                  plot_height=300, 
                                title="Rome, Italy",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
rome_air_temp_and_fit.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
rome_air_temp_and_fit.line(x=rome_data["air_temp"], y=rome_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


stuttgart_air_temp_and_fit = figure(plot_width=600, 
                                    plot_height=300, 
                                title="Stuttgart, Germany",
                                y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range)
stuttgart_air_temp_and_fit.circle(x="air_temp", y="Total_consumption", legend_label="Total Heat Consumption", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
stuttgart_air_temp_and_fit.line(x=stuttgart_data["air_temp"], y=stuttgart_regline, legend_label="RMSE", color="yellow", alpha=1, line_width=2.5)


all_europe = figure(plot_width=1200, 
                    plot_height=300, 
                    title="Power vs Air Temp",)
all_europe.circle(x="air_temp", y="Total_consumption", legend_label="London, UK", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
all_europe.circle(x="air_temp", y="Total_consumption", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
all_europe.circle(x="air_temp", y="Total_consumption", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
all_europe.circle(x="air_temp", y="Total_consumption", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)



show(layout([
    [all_europe],
    [london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],
    [rome_air_temp_and_fit, madrid_air_temp_and_fit],

]))

## Load Duration Curve for Europe

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["Total_consumption"]
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["Total_consumption"]
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["Total_consumption"]
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["Total_consumption"]

x = np.array(range(1,london_data.shape[0] + 1))
x_max = max(x)
x_cumsum = x * 100 / x_max 

y_london = london_data.sort_values(ascending=False)
y_london_max = london_data.max()
y_london_norm = y_london * 100 / y_london_max


y_stuttgart = stuttgart_data.sort_values(ascending=False)
y_stuttgart_max = stuttgart_data.max()
y_stuttgart_norm = y_stuttgart * 100 / y_stuttgart_max


y_rome = rome_data.sort_values(ascending=False)
y_rome_max = rome_data.max()
y_rome_norm = y_rome * 100 / y_rome_max


y_madrid = madrid_data.sort_values(ascending=False)
y_madrid_max = madrid_data.max()
y_madrid_norm = y_madrid * 100 / y_madrid_max

max_across_europe = max(y_london_max, y_stuttgart_max, y_rome_max, y_madrid_max,)
y_london_norm_all =  y_london * 100 / max_across_europe
y_stuttgart_norm_all =  y_stuttgart * 100 / max_across_europe
y_rome_norm_all =  y_rome * 100 / max_across_europe
y_madrid_norm_all =  y_madrid * 100 / max_across_europe

load_duration_curve_europe_y_equal_x_equal = figure(plot_width=600, 
                                plot_height=300, 
                                title="LDC for Europe",
                                x_range=(0,9000))
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_london, legend_label="London ", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_stuttgart, legend_label="Stuttgart ", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_rome, legend_label="Rome ", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.line(x=x, y=y_madrid, legend_label="Madrid ", color=color_map["madrid"], alpha=1, line_width=2.5)

load_duration_curve_europe_y_equal_x_norm = figure(plot_width=600, 
                                plot_height=300, 
                                title="LDC for Europe",
                                x_range=(0,105))
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_london, legend_label="London ", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_stuttgart, legend_label="Stuttgart ", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_rome, legend_label="Rome ", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.line(x=x_cumsum, y=y_madrid, legend_label="Madrid ", color=color_map["madrid"], alpha=1, line_width=2.5)


load_duration_curve_europe_y_norm_x_equal = figure(plot_width=600, 
                                plot_height=300,
                                title="LDC for Europe",
                                x_range=(0,9000))
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_london_norm_all, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_stuttgart_norm_all, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_rome_norm_all, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.line(x=x, y=y_madrid_norm_all, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


load_duration_curve_europe_y_norm_x_norm = figure(plot_width=600, 
                                plot_height=300,
                                title="LDC for Europe",
                                x_range=(0,105))
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_london_norm_all, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_stuttgart_norm_all, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_rome_norm_all, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.line(x=x_cumsum, y=y_madrid_norm_all, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


show(layout([
    [load_duration_curve_europe_y_equal_x_equal, load_duration_curve_europe_y_equal_x_norm],
    [load_duration_curve_europe_y_norm_x_equal, load_duration_curve_europe_y_norm_x_norm],
]))



In [ ]:
load_duration_curve_europe_norm_y_norm_x_equal = figure(plot_width=1200, 
                                plot_height=600, 
                                title="LDC for Europe",
                                x_range=(0,9000))
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.line(x=x, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


load_duration_curve_europe_norm_y_norm_x_norm = figure(plot_width=1200, 
                                plot_height=600, 
                                title="LDC for Europe",
                                x_range=(0,105))
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_cumsum, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)



#load_duration_curve_europe_norm = figure(plot_width=1200, 
#                                plot_height=600, 
#                                title="LDC for Europe",
 #                               x_range=(0,9000))
#load_duration_curve_europe_norm.line(x=x, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)



#load_duration_curve_europe_norm = figure(plot_width=1200, 
#                                plot_height=600, 
#                                title="LDC for Europe",
#                                x_range=(0,105))
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_london_norm, legend_label="London Norm", color=color_map["london"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_stuttgart_norm, legend_label="Stuttgart Norm", color=color_map["stuttgart"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_rome_norm, legend_label="Rome Norm", color=color_map["rome"], alpha=1, line_width=2.5)
#load_duration_curve_europe_norm.line(x=x_cumsum, y=y_madrid_norm, legend_label="Madrid Norm", color=color_map["madrid"], alpha=1, line_width=2.5)


show(layout([
    [load_duration_curve_europe_norm_y_norm_x_equal],[load_duration_curve_europe_norm_y_norm_x_norm],
    #[load_duration_curve_europe_y_norm_x_equal, load_duration_curve_europe_y_norm_x_norm],
]))



In [ ]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show


def make_plot(title, hist, edges, x, pdf=None, cdf=None):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, 
           bottom=0, 
           left=edges[:-1], 
           right=edges[1:],
           fill_color="navy", 
           line_color="white", 
           alpha=0.5)
    p.y_range.start = 0
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    #p.grid.grid_line_color="white"
    return p

# Normal Distribution

#mu, sigma = 0, 0.5

#measured = np.random.normal(mu, sigma, 1000)
#hist, edges = np.histogram(measured, density=True, bins=50)

measured = london_data.sort_values(ascending=False)
hist, edges = np.histogram(measured, density=False, bins=50)

#x = np.linspace(-2, 2, 1000)
#pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
#cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1 = make_plot("Normal Distribution (μ=0, σ=0.5)", hist, edges, x)
show(layout([
    [p1]
]))

### Whole year

In [ ]:
visualize_on_same_row(data_list=[miami[["space_heating_temp","hot_water_temp"]],
                                 fresno[["space_heating_temp","hot_water_temp"]],
                                 olympia[["space_heating_temp","hot_water_temp"]],
                                 rochester[["space_heating_temp","hot_water_temp"]],
                                 ospitaletto[["space_heating_temp","hot_water_temp"]]],
                      ylabel="Temperature in Celsius", 
                      xlabel="Date",
                      suptitle='Calculated Space Heating and Hot Water temperatures')

In [ ]:
month = int(input("Enter month to analyze (values between 1 to 12): "))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.month == month ]
november_data[["space_heating_temp","hot_water_temp"]].plot(figsize=(10,6), color=["#4e97a2",'#82272e'])

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
Tamb_hourly["user_temp"].plot(figsize=(10,6))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
november_data["Tuser"].plot(figsize=(10,6))

## Visualization of Net temperature

In [ ]:
Tamb_hourly[["net_temp"]][Tamb_hourly.index < "2010-01-31"].plot()

## Visualization of COP

In [ ]:
Tamb_hourly[['COP']].plot(figsize=(20,6))

## Visualization of Energy Loss

In [ ]:
Tamb_hourly[['E_loss_tot']].plot(figsize=(20,6))

## Space Heating Distribution Plot

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,10))

# Stuttgart

sns.lineplot(data=stuttgart[["space_heating_dist"]], ax=ax1, color='#4d4dff')
ax1.set_title('Stuttgart')
ax1.set_ylabel('Space heating hourly distribution')
ax1.grid()

# London
sns.lineplot(data=london[["space_heating_dist"]], ax=ax2, color='#8686ef')
ax2.set_title('London')
ax2.grid()

# Madrid
sns.lineplot(data=madrid[["space_heating_dist"]], ax=ax3, color='#ef8e49')
ax3.set_title('Madrid')
ax3.set_ylabel('Space heating hourly distribution')
ax3.grid()

# Rome
sns.lineplot(data=rome[["space_heating_dist"]], ax=ax4, color= '#ca0c6e')
ax4.set_title('Rome')
ax4.grid()

fig.tight_layout()
plt.show()